## 2b. Calculate closed-system degassing path when there is melt and vapor at the start of calculation

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations. You need to install VolFe once on your machine, if you haven't yet, uncomment the line below (remove the #).

In [ ]:
# Install VolFe on your machine. Remove the # from line below to do this (don't remove the # from this line!).
# pip install VolFe

In [1]:
# import python packages
import pandas as pd
import gmpy2 as gp
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

Sometimes we do not know the bulk composition of the system in terms of dissolved volatile content of the melt and an fO2 estimate at Pvsat. 
Instead, we might know the melt composition some way along the degassing path (e.g., from melt inclusion analysis) that was in equilibrium with a vapor and an idea of how much vapor was present.
We can still calculate a closed-system degassing path if we add together the melt and vapor composition.

So instead of Sari15-04-33 from Brounce et al. (2014) being representative of the bulk composition of the system, let's instead say its a melt composition some way along the degassing path (still assuming a temperature of 1200 'C). 
We estimate that the initial melt contained 2. wt% CO2-eq, which we add to the input dataframe under "initial_CO2wtpc".

In [8]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Sari15-04-33',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 47.89, # wt%
           'TiO2': 0.75, # wt%
           'Al2O3': 16.74, # wt%
           'FeOT': 9.43, # wt%
           'MnO': 0.18, # wt%
           'MgO': 5.92, # wt%
           'CaO': 11.58, # wt%
           'Na2O': 2.14, # wt%
           'K2O': 0.63, # wt%
           'P2O5': 0.17, # wt%
           'H2O': 4.17, # wt%
           'CO2ppm': 1487., # ppm
           'STppm': 1343.5, # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.195,
           'initial_CO2wtpc': 2.} # initial CO2 content of the system in wt%

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

To use the initial CO2 in the calculation, we have to change the "bulk_composition" option to "CO2" so it knows how to calculate the bulk composition.
Also, we'll run the calculation from given composition rather than the calculated bulk composition (currently these types of calculations cannot be started from the true bulk composition).
This means changing "starting_P" to "measured".

In [9]:
# choose the options I want - everything else will use the default options
my_models = [['bulk_composition','CO2'],['starting_P','measured']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

VolFe will then calculate the bulk composition of the system by calculating the melt and vapor composition at Pvsat based on the input composition, and then add that vapor composition to the melt such that the bulk composition is 1 wt% CO2.

The calculation below will run with the default pressure step size of 10 bar so it might take a while to run.

In [10]:
vf.calc_gassing(my_analysis, models=my_models)

Sari15-04-33
1200.0 3863.5831334191316 2024-04-06 21:56:16.441737
3830.0 : Switching solve species from OCS to OCH
1200.0 2860.0 0.7054322413478316 WARNING: sulfide-saturated 2024-04-06 21:58:04.556908
1200.0 1860.0 0.6541850087854018  2024-04-06 21:59:16.091505
1200.0 860.0 0.5437825609416658  2024-04-06 22:00:20.645155
180.0 : Switching solve species from OCH to OHS
110.0 : Switching solve species from OHS to OCS
10.0 : Switching solve species from OCS to OCH
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan en

,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,Sari15-04-33,1200.0,3863.583133,47.948689,0.750919,16.760515,9.441557,0.180221,5.927255,11.594191,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:56:16.571847
0,Sari15-04-33,1200.0,3850.0,47.947549950587188,0.75090128341909423,16.760116645914167,9.4413321368560759,0.18021630802058253,5.9271141304547115,11.593915815990812,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:56:20.695606
0,Sari15-04-33,1200.0,3840.0,47.9476455802332,0.75090278106441666,16.760150073357778,9.4413509672499316,0.18021666745545994,5.9271259518684616,11.593938939634583,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:56:21.630787
0,Sari15-04-33,1200.0,3830.0,47.947743277882353,0.75090431109650879,16.760184223674059,9.4413702048534276,0.18021703466316197,5.9271380289217674,11.593962563330081,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:57:04.770223
0,Sari15-04-33,1200.0,3820.0,47.94784137701879,0.75090584741624777,16.76021851433066,9.4413895215136279,0.18021740337989955,5.927150155605581,11.593986284106862,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:57:05.346431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Sari15-04-33,1200.0,60.0,48.013362758186815,0.75193197052913252,16.783121582210214,9.4542913094529553,0.18046367292699173,5.9352496873766114,11.609829624969791,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:02:35.046941
0,Sari15-04-33,1200.0,50.0,48.013623447635268,0.75193605315778811,16.783212706481834,9.4543426417039171,0.18046465275786905,5.9352819129254755,11.609892660756245,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:02:36.408729
0,Sari15-04-33,1200.0,40.0,48.013909723642769,0.75194053649471826,16.78331277456212,9.4543990121935906,0.18046572875873257,5.9353173013983138,11.609961883478462,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:02:37.937352
0,Sari15-04-33,1200.0,30.0,48.014234994333833,0.75194563052308094,16.783426473275171,9.4544630611102036,0.18046695132553947,5.9353575102621861,11.610040535276369,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:02:39.289163


We might not have an estimate of the initial CO2 content of the melt.
Instead we might estimate that there is 1 wt% vapor present in equilibrium with the known melt composition.
In that case, we specify the amount of vapor present as wt% of the system instead of the initial CO2 content in the input data frame.

In [11]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Sari15-04-33',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 47.89, # wt%
           'TiO2': 0.75, # wt%
           'Al2O3': 16.74, # wt%
           'FeOT': 9.43, # wt%
           'MnO': 0.18, # wt%
           'MgO': 5.92, # wt%
           'CaO': 11.58, # wt%
           'Na2O': 2.14, # wt%
           'K2O': 0.63, # wt%
           'P2O5': 0.17, # wt%
           'H2O': 4.17, # wt%
           'CO2ppm': 1487., # ppm
           'STppm': 1343.5, # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.195,
           'wt_g': 1.} # wt% vapor in equilibrium with the melt

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

To use the amount of vapor present in the calculation, we have to change the "bulk_composition" option to "wtg" and keep "starting_P" as "measured".

In [12]:
# choose the options I want - everything else will use the default options
my_models = [['bulk_composition','wtg'],['starting_P','measured']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

VolFe will then calculate the bulk composition of the system by calculating the melt and vapor composition at Pvsat based on the input composition, and then add 1 wt% of that vapor composition to the melt.

Again, the calculation below will run with the default pressure step size of 10 bar so it might take a while to run.

In [13]:
vf.calc_gassing(my_analysis, models=my_models)

Sari15-04-33
1200.0 3863.5831334191316 2024-04-06 22:09:31.319748
3710.0 : Switching solve species from OCS to OCH
1200.0 2860.0 0.7203326398545071 WARNING: sulfide-saturated 2024-04-06 22:11:24.954842
1200.0 1860.0 0.6733372293076361  2024-04-06 22:12:26.112943
1200.0 860.0 0.5459765836172341  2024-04-06 22:13:29.038874
160.0 : Switching solve species from OCH to OHS
160.0 : Switching solve species from OHS to OCS
1.0 : Switching solve species from OCS to OCH
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan encountered
nan enc

,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,Sari15-04-33,1200.0,3863.583133,47.948689,0.750919,16.760515,9.441557,0.180221,5.927255,11.594191,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:09:31.458705
0,Sari15-04-33,1200.0,3850.0,47.948119570509562,0.75091020417377674,16.760315757158708,9.4414443004782811,0.18021844900170647,5.927184544945014,11.594053552443114,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:09:32.261133
0,Sari15-04-33,1200.0,3840.0,47.948175611772854,0.75091108182980992,16.760335346441366,9.4414553355401551,0.1802186596391544,5.9271914725766379,11.594067103452275,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:09:33.045709
0,Sari15-04-33,1200.0,3830.0,47.948232740895037,0.7509119765226826,16.760355315986288,9.4414665848118631,0.18021887436544382,5.9271985346857079,11.594080917510212,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:09:33.616648
0,Sari15-04-33,1200.0,3820.0,47.948290221654986,0.75091287672251461,16.760375408446521,9.4414779033244116,0.18021909041340356,5.9272056402630469,11.594094816595629,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:09:34.197016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Sari15-04-33,1200.0,50.0,48.013743445513001,0.75193793243129559,16.783254651866514,9.454366270436168,0.18046510378351099,5.9352967466576985,11.609921676739205,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:15:54.187270
0,Sari15-04-33,1200.0,40.0,48.01402401833468,0.75194232645126391,16.783352726392195,9.454421517913886,0.18046615834830337,5.9353314301219768,11.60998952040751,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:15:56.350057
0,Sari15-04-33,1200.0,30.0,48.01434012187957,0.75194727691396324,16.783463220719653,9.4544837617315522,0.18046734645935114,5.9353705057742161,11.61006595555159,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:15:58.652035
0,Sari15-04-33,1200.0,20.0,48.01471639929963,0.75195316975307414,16.783594748888589,9.4545578543619797,0.18046876074073781,5.935417019917594,11.610156940987466,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 22:16:09.507138


### in progress ###
Plotting...
